In [ ]:
import pandas as pd
import requests

In [ ]:
# 1. URL de exemplo de uma API ou fonte pública
DATA_URL = "https://api.portaldatransparencia.gov.br/api/v1/some-dataset"
API_KEY = "SUA_API_KEY"  # Substitua pela sua chave da API, se necessário

In [ ]:
# 2. Função para baixar os dados
def fetch_data(url, headers=None):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Levanta erro se houver problema
        return pd.DataFrame(response.json())
    except Exception as e:
        print(f"Erro ao buscar os dados: {e}")
        return pd.DataFrame()


In [ ]:
# 3. Função para validar os dados
def validate_data(df):
    print("\n===== Validação de Dados =====")

    # Verificar tamanho do dataset
    print(f"Quantidade de linhas: {len(df)}")

    # Verificar valores ausentes
    missing_values = df.isnull().sum()
    print("\nValores ausentes por coluna:")
    print(missing_values[missing_values > 0])

    # Verificar duplicados
    duplicates = df.duplicated().sum()
    print(f"\nLinhas duplicadas: {duplicates}")

    # Verificar consistência de campos numéricos
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    print("\nResumo estatístico de campos numéricos:")
    print(df[numeric_cols].describe())


In [ ]:
# 4. Função para verificar a última atualização
def check_last_update(df, date_column):
    if date_column in df.columns:
        df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
        last_date = df[date_column].max()
        print(f"\nÚltima data registrada: {last_date}")
        if (datetime.now() - last_date).days > 30:
            print("ALERTA: Dados podem estar desatualizados!\n")
        else:
            print("Os dados estão atualizados.\n")
    else:
        print("Coluna de data não encontrada para validação.")


In [ ]:
# 5. Executar tudo

# Configurar cabeçalhos da API, se necessário
headers = {"Authorization": f"Bearer {API_KEY}"} if API_KEY else None

# Baixar os dados
print("Baixando dados...")
data = fetch_data(DATA_URL, headers)

if not data.empty:
    # Validar os dados
    validate_data(data)

    # Verificar a última atualização
    check_last_update(data, date_column="dataAtualizacao")  # Substitua pelo nome real da coluna de data

    # Salvar dados para uso posterior
    data.to_csv("dados_validos.csv", index=False)
    print("\nDados salvos como 'dados_validos.csv'.")
else:
    print("Nenhum dado foi baixado.")
